<div class="alert alert-block alert-success">  
    <center><h2><strong>👨‍💻 Getting Started with RSNA-MICCAI Brain Tumor Radiogenomic Classification</strong></h2></center>
    <i></i>
</div>

![](https://www.mdpi.com/jcm/jcm-10-01411/article_deploy/html/images/jcm-10-01411-g001.png)

### Take overview of the Competition here 

https://www.kaggle.com/c/rsna-miccai-brain-tumor-radiogenomic-classification/overview

### The exact mpMRI scans included are:

* Fluid Attenuated Inversion Recovery
* T1-weighted pre-contrast (T1w)
* T1-weighted post-contrast (T1Gd)
* T2-weighted (T2)


### Files
train/ - folder containing the training files, with each top-level folder representing a subject
train_labels.csv - file containing the target MGMT_value for each subject in the training data (e.g. the presence of MGMT promoter methylation)
test/ - the test files, which use the same structure as train/; your task is to predict the MGMT_value for each subject in the test data. NOTE: the total size of the rerun test set (Public and Private) is ~5x the size of the Public test set
sample_submission.csv - a sample submission file in the correct format

### Read the refrence paper for more ideas 

U.Baid, et al., “The RSNA-ASNR-MICCAI BraTS 2021 Benchmark on Brain Tumor Segmentation and Radiogenomic Classification”, arXiv:2107.02314, 2021.
https://arxiv.org/abs/2107.02314

### Importing Libraries

In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pydicom as dicom
import cv2
import ast

import warnings
warnings.filterwarnings("ignore")

### Path of dataset

In [ ]:
path = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/'
os.listdir(path)

#### Loading dataset

In [ ]:
train_data = pd.read_csv(path+'train_labels.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

### Overview of dataset

In [ ]:
print('Samples train:', len(train_data))
print('Samples test:', len(samp_subm))

In [ ]:
train_data.head()

### MGMT_value counts 

In [ ]:
train_data["MGMT_value"].value_counts().head(2).plot(kind = 'pie', autopct='%1.1f%%', figsize=(8, 8)).legend()

In [ ]:
train_data["MGMT_value"].value_counts()

In [ ]:
samp_subm.head()

# Read Dicom Files
We consider the first train sample.
```
Training/Validation/Testing
│
└─── 00000
│   │
│   └─── FLAIR
│   │   │ Image-1.dcm
│   │   │ Image-2.dcm
│   │   │ ...
│   │   
│   └─── T1w
│   │   │ Image-1.dcm
│   │   │ Image-2.dcm
│   │   │ ...
│   │   
│   └─── T1wCE
│   │   │ Image-1.dcm
│   │   │ Image-2.dcm
│   │   │ ...
│   │   
│   └─── T2w
│   │   │ Image-1.dcm
│   │   │ Image-2.dcm
│   │   │ .....
│   
└─── 00001
│   │ ...
│   
│ ...   
│   
└─── 00002
│   │ ...
```

### Extract folder id of the first train sample

In [ ]:
folder = str(train_data.loc[0, 'BraTS21ID']).zfill(5)
folder

### Folders content

In [ ]:
os.listdir(path+'train/'+folder)

In [ ]:
print('Number of FLAIR images:', len(os.listdir(path+'train/'+folder+'/'+'FLAIR')))
print('Number of T1w images:', len(os.listdir(path+'train/'+folder+'/'+'T1w')))
print('Number of T1wCE images:', len(os.listdir(path+'train/'+folder+'/'+'T1wCE')))
print('Number of T2w images:', len(os.listdir(path+'train/'+folder+'/'+'T2w')))

### Read image

In [ ]:
path_file = ''.join([path, 'train/', folder, '/', 'FLAIR/'])
image = os.listdir(path_file)[0]
data_file = dicom.dcmread(path_file+image)
img = data_file.pixel_array

### Image shape

In [ ]:
print('Image shape:', img.shape)

> # Flair Images 

In [ ]:
def plot_examples(row = 0, cat = 'FLAIR'): 
    folder = str(train_data.loc[row, 'BraTS21ID']).zfill(5)
    path_file = ''.join([path, 'train/', folder, '/', cat, '/'])
    images = os.listdir(path_file)
    
    fig, axs = plt.subplots(1, 5, figsize=(30, 30))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    
    for num in range(5):
        data_file = dicom.dcmread(path_file+images[num])
        img = data_file.pixel_array
        axs[num].imshow(img, cmap='gray')
        axs[num].set_title(cat+' '+images[num])
        axs[num].set_xticklabels([])
        axs[num].set_yticklabels([])
        
row = 0
plot_examples(row = row, cat = 'FLAIR')

> # T1w Images 

In [ ]:
plot_examples(row = row, cat = 'T1w')

> # T1wCE Images 

In [ ]:
plot_examples(row = row, cat = 'T1wCE')

> # T2w Images 

In [ ]:
plot_examples(row = row, cat = 'T2w')

# Transfer Learning for Brain Tumor Radiogenomic Classification¶

![](https://www.researchgate.net/profile/Max_Ferguson/publication/322512435/figure/fig3/AS:697390994567179@1543282378794/Fig-A1-The-standard-VGG-16-network-architecture-as-proposed-in-32-Note-that-only.png)

In [ ]:
VGG_types = {
    'VGG11' : [64, 'M', 128, 'M', 256, 256, 'M', 512,512, 'M',512,512,'M'],
    'VGG13' : [64,64, 'M', 128, 128, 'M', 256, 256, 'M', 512,512, 'M', 512,512,'M'],
    'VGG16' : [64,64, 'M', 128, 128, 'M', 256, 256,256, 'M', 512,512,512, 'M',512,512,512,'M'],
    'VGG19' : [64,64, 'M', 128, 128, 'M', 256, 256,256,256, 'M', 512,512,512,512, 'M',512,512,512,512,'M']
}
class VGGnet(nn.Module):
    def __init__(self, model, in_channels=3, num_classes=10, init_weights=True):
        super(VGGnet,self).__init__()
        self.in_channels = in_channels

        # create conv_layers corresponding to VGG type
        self.conv_layers = self.create_conv_laters(VGG_types[model])

        self.fcs = nn.Sequential(
            nn.Linear(1536, 1536//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1536//2, 1536//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1536//2, num_classes),
        )

        # weight initialization
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fcs(x)
        return x

    # defint weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    # define a function to create conv layer taken the key of VGG_type dict 
    def create_conv_laters(self, architecture):
        layers = []
        in_channels = self.in_channels # 3

        for x in architecture:
            if type(x) == int: # int means conv layer
                out_channels = x

                layers += [nn.Conv3d(in_channels=in_channels, out_channels=out_channels,
                                     kernel_size=(3,2,3), stride=(1,1,1), padding=(1,1,1)),
                           nn.BatchNorm3d(x),
                           nn.ReLU()]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool3d(kernel_size=(2,2,2), stride=(2,2,2))]
        
        return nn.Sequential(*layers)

In [ ]:
# define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# creat VGGnet object
# Choose between 'VGG11', 'VGG13', 'VGG16', 'VGG19'
model = VGGnet('VGG16', in_channels=1, num_classes=1, init_weights=True).to(device)
print(model)

## More code is coming Soon. Please upvote if you like the work and you will get notifications with additions. 

<center><img src="https://thumbs.gfycat.com/AshamedWeightyDachshund-max-1mb.gif"></center>

<div class="alert alert-block alert-info">  
    <h3><strong>Guideline to submit the results</strong></h3>
    <i></i>
</div>

> **Check the submission.csv file so we will know the format that how can submit the predictions. Before we do so, it's worth reminding ourselves that this is a code-only competition, meaning that your submission file has to be generated in a script/notebook. The submission.csv file demonstrated what kind of file needs to be produced:**

In [ ]:
samp_subm.head()

#### How to submit results 

In [ ]:
samp_subm.to_csv('submission.csv', index=False)

> **We submitted the predictions but we will come back soon with model training and a lot more.**

<div class="alert alert-block alert-danger">  
<h1>If you like my work, please upvote ^ 👍 my code so that i will be motivated to share more valuable and helpful work on this competition 😍</h1>
        </p>
</div>